In [45]:
from datetime import datetime
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
import networkx as nx
from networkx.algorithms import community

from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

os.environ["OPENAI_API_KEY"] = ''

In [3]:
import pandas as pd

file_path = 'nutrimuscle_combined_reviews.xlsx'
df_review = pd.read_excel(file_path)

# Now, you can work with the DataFrame `df_review` to analyze and manipulate the data
df_review

,title,Text,rating,date_created,Source,review_country,lemmatized_review,Aspects,Entities
0,Rien,ProblÃ¨me de poids,5,2022-07-21 00:00:00+00:00,Amazon,fr,poid,NaN,NaN
1,Produit conforme aux attentes contrairement au...,Omega 3 de qualite,5,2021-06-25 00:00:00+00:00,Amazon,fr,omega qualite,NaN,NaN
2,QualitÃ©,Excellent.,5,2022-04-26 00:00:00+00:00,Amazon,fr,excellent,{'product': 1},NaN
3,Excellent,Excellent,5,2021-06-05 00:00:00+00:00,Amazon,fr,excellent,{'product': 1},NaN
4,Conforme,Top qualitÃ©,5,2021-05-24 00:00:00+00:00,Amazon,fr,top qualitÃ©,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2576,Je n'ai reÃ§u aucun produit.,Je n'ai reÃ§u aucun produit.,1,2023-03-27 11:46:48+00:00,Trustpilot,NaN,n'ai produit,NaN,NaN
2577,Nutrimuscle au top ðŸ‘Œ,Nutrimuscle au top ðŸ‘Œ,5,2022-10-24 20:28:50+00:00,Trustpilot,NaN,nutrimuscle au top ðŸ‘Œ,NaN,NaN
2578,Merci beaucoup,Merci beaucoup,5,2022-10-22 18:54:19+00:00,Trustpilot,NaN,merci beaucoup,NaN,NaN
2579,OK mec OK ok,"OK, mec t'as ok ok ok ok ok",5,2023-01-18 03:13:42+00:00,Trustpilot,NaN,t'as,NaN,NaN


In [16]:
# Extract the 'Text' column (assuming the column name is 'Text')
txt = df_review['Text']

# Initialize an empty list to store the segments
segments = []

# Iterate through the text data, splitting by period, then by comma
for text in txt:
    for segment in text.split('.'):
        segments += segment.split(',')

# You can optionally add punctuation back in if needed
# segments = [segment + '.' for segment in segments]


In [30]:
def create_sentences(segments, MIN_WORDS, MAX_WORDS):
    sentences = []
    is_new_sentence = True
    sentence_length = 0
    sentence_num = 0
    sentence_segments = []

    for i in range(len(segments)):
        segment = segments[i].strip() # Remove leading/trailing whitespace
        if segment == '':
            continue # Skip empty segments

        if is_new_sentence:
            is_new_sentence = False
        # Append the segment
        sentence_segments.append(segment)
        segment_words = segment.split(' ')
        sentence_length += len(segment_words)
        
        # If exceed MAX_WORDS, then stop at the end of the segment
        # Only consider it a sentence if the length is at least MIN_WORDS
        if (sentence_length >= MIN_WORDS and segment[-1] == '.') or sentence_length >= MAX_WORDS:
            sentence = ' '.join(sentence_segments)
            sentences.append({
                'sentence_num': sentence_num,
                'text': sentence,
                'sentence_length': sentence_length
            })
            # Reset
            is_new_sentence = True
            sentence_length = 0
            sentence_segments = []
            sentence_num += 1

    return sentences


def create_chunks(sentences, CHUNK_LENGTH, STRIDE):

  sentences_df = pd.DataFrame(sentences)
  
  chunks = []
  for i in range(0, len(sentences_df), (CHUNK_LENGTH - STRIDE)):
    chunk = sentences_df.iloc[i:i+CHUNK_LENGTH]
    chunk_text = ' '.join(chunk['text'].tolist())
    
    chunks.append({
      'start_sentence_num': chunk['sentence_num'].iloc[0],
      'end_sentence_num': chunk['sentence_num'].iloc[-1],
      'text': chunk_text,
      'num_words': len(chunk_text.split(' '))
    })
    
  chunks_df = pd.DataFrame(chunks)
  return chunks_df.to_dict('records')

def parse_title_summary_results(results):
  out = []
  for e in results:
    e = e.replace('\n', '')
    if '|' in e:
      processed = {'title': e.split('|')[0],
                    'summary': e.split('|')[1][1:]
                    }
    elif ':' in e:
      processed = {'title': e.split(':')[0],
                    'summary': e.split(':')[1][1:]
                    }
    elif '-' in e:
      processed = {'title': e.split('-')[0],
                    'summary': e.split('-')[1][1:]
                    }
    else:
      processed = {'title': '',
                    'summary': e
                    }
    out.append(processed)
  return out

In [144]:
sentences = create_sentences(segments, MIN_WORDS=5, MAX_WORDS=80)

# for sentence in sentences:
#     print(sentence['text'])

In [146]:
len(sentences)

634

In [40]:
sentences = create_sentences(segments, MIN_WORDS=20, MAX_WORDS=80)
chunks = create_chunks(sentences, CHUNK_LENGTH=5, STRIDE=1)
chunks_text = [chunk['text'] for chunk in chunks]

In [114]:
len(chunks_text)

159

In [116]:
#view last chunck contains the last text reviews in the data
chunks_text[158]

"bah jâ€™ai rien reÃ§u Ho provato a contattarli piÃ¹ volte ma niente da fare Pessima assistenza Sono sempre molto soddisfatto dei prodotti di nutrimuscle Alta qualitÃ\xa0 sempre garantita Excellente prestation Ho ordinato 500 g di citrullina e mi Ã¨ arrivato un pacco contenente delle proteine wheyâ€¦ Ho effettuato il reso seguendo tutte le loro istruzioni e mi Ã¨ semplicemente arrivata une e-mail con scritto che hanno ricevuto il reso e che riceverÃ³ un rimborso entro 14 giorni lavorativi senza specificare neanche in che modo viene effettuato questo reso Inoltre assistenza telefonica inesistente chiamando il numero che ti forniscono non risponde nessun operatore addirittura ultimamente non parte neanche piÃ¹ la chiamata Super cordiales service rapide miam miam la bonne schnouille Prodotto ottimo e spedizione impeccabile Professionnel avis satisfaisant Snelle levering Zelfs zonder kennis france kon ik makkelijk product bestellen Heel tevreden ðŸ˜Š Aucun arrangement bien bien bien Nickel

In [131]:
from datetime import datetime
from collections import defaultdict

def summarize_stage_1(chunks_text):
  
  def limit_words(text, max_words=100):
    words = text.split()
    return ' '.join(words[:max_words])

  print(f'Start time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
  
  map_prompt_template = """Summarize the following text, and provide a general theme or topic (maximum 8 words) for which the text is focused. Break down the comments by positive and negative aspects around the theme:
  {text}

  Return your answer in the following format:
  Theme/Topic | Positive Comments: Positive comments here | Negative Comments: Negative comments here
  e.g. 
  Customer Service | Positive Comments: Great support and helpful staff | Negative Comments: Long waiting times and unresponsive support"""

  map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

  # Define the LLMs
  map_llm = OpenAI(temperature=0, model_name='text-davinci-003')
  map_llm_chain = LLMChain(llm=map_llm, prompt=map_prompt)
  map_llm_chain_input = [{'text': t} for t in chunks_text]
  # Run the input through the LLM chain
  map_llm_chain_results = map_llm_chain.apply(map_llm_chain_input)

  # Parsing the theme, positive and negative comments
  themes_and_comments = [{'theme': e['text'].split('|')[0].strip(), 'positive': limit_words(e['text'].split('|')[1].strip().replace("Positive Comments:", "").strip()), 'negative': limit_words(e['text'].split('|')[2].strip().replace("Negative Comments:", "").strip())} for e in map_llm_chain_results]

  # Collect themes and comments
  theme_counts = defaultdict(int)
  theme_positives = defaultdict(list)
  theme_negatives = defaultdict(list)
  for item in themes_and_comments:
    theme = item['theme']
    positive = item['positive']
    negative = item['negative']
    theme_counts[theme] += 1
    theme_positives[theme].append(positive)
    theme_negatives[theme].append(negative)

  # Compile the results
  result = []
  for theme, count in theme_counts.items():
    positives = ' '.join([limit_words(p) for p in theme_positives[theme]])  # Limit to 100 words
    negatives = ' '.join([limit_words(n) for n in theme_negatives[theme]])  # Limit to 100 words
    result.append({
      'theme': theme,
      'count': count,
      'positive_comments': positives,
      'negative_comments': negatives
    })

  print(f'Stage 1 done time {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
  return result


In [133]:
stage_1_outputs = summarize_stage_1(chunks_text)
stage_1_themes = [e['theme'] for e in stage_1_outputs]
stage_1_counts = [e['count'] for e in stage_1_outputs]
stage_1_positive_comments = [e['positive_comments'] for e in stage_1_outputs]
stage_1_negative_comments = [e['negative_comments'] for e in stage_1_outputs]

Start time: 2023-08-09 16:01:57
Stage 1 done time 2023-08-09 16:02:20


In [137]:
stage_1_outputs

[{'theme': 'Omega 3 Quality',
  'count': 1,
  'positive_comments': 'Top quality, authentic, good, easy to take, no aftertaste, satisfactory',
  'negative_comments': 'Expensive, difficulty sleeping, maux de tête, no positive effects, large capsules, fishy smell'},
 {'theme': 'Omega 3 Supplements',
  'count': 1,
  'positive_comments': 'Quality product, good value, no fishy taste, good packaging, beneficial for health',
  'negative_comments': 'Expensive, large capsules, not always accurate information'},
 {'theme': 'Omega 3',
  'count': 1,
  'positive_comments': 'High quality, easy to swallow, beneficial for health',
  'negative_comments': 'Unclear origin of fish, half-filled pot, large size of capsule'},
 {'theme': 'Omega-3 Supplements',
  'count': 1,
  'positive_comments': 'Easy to swallow, good quality, effective relief, halal and kosher',
  'negative_comments': 'Expensive, large capsules, too many capsules to take, no taste, no results yet'},
 {'theme': 'Collagen Supplement',
  'count

In [138]:
# Sort by count in descending order
top_5_themes = sorted(stage_1_outputs, key=lambda x: x['count'], reverse=True)[:5]

# Extract the themes, counts, and general summaries for the top 5 themes
top_5_theme_names = [e['theme'] for e in top_5_themes]
top_5_counts = [e['count'] for e in top_5_themes]
#top_5_general_summaries = [e['general_summary'] for e in top_5_themes]

pd.DataFrame(top_5_themes)


,theme,count,positive_comments,negative_comments
0,Nutrimuscle Quality,34,"Quality products, good advice, fast delivery, ...","Expensive, low dosage, uncertain production Po..."
1,Product Quality,27,"Quality products, fast delivery, bonus gifts Q...","Incomplete product information, high prices, p..."
2,Nutrimuscle,17,"Efficient, quality product, helpful for balanc...","Not a miracle product, not as effective as oth..."
3,Quality & Delivery,15,"Good quality products, fast delivery, good pac...","Expensive delivery, incomplete orders, unrelia..."
4,Nutrimuscle Products,11,"Great products, timely delivery, quality, tran...","Poor packaging, delayed delivery, bad taste of..."


In [140]:
df_all_themes = pd.DataFrame(stage_1_outputs)

df_all_themes.sort_values("count", ascending=False)

,theme,count,positive_comments,negative_comments
7,Nutrimuscle Quality,34,"Quality products, good advice, fast delivery, ...","Expensive, low dosage, uncertain production Po..."
12,Product Quality,27,"Quality products, fast delivery, bonus gifts Q...","Incomplete product information, high prices, p..."
6,Nutrimuscle,17,"Efficient, quality product, helpful for balanc...","Not a miracle product, not as effective as oth..."
13,Quality & Delivery,15,"Good quality products, fast delivery, good pac...","Expensive delivery, incomplete orders, unrelia..."
11,Nutrimuscle Products,11,"Great products, timely delivery, quality, tran...","Poor packaging, delayed delivery, bad taste of..."
14,Quality of Products,9,"Excellent quality, good miscibility, fast deli...","Package arrived broken, difficult to dissolve,..."
23,Quality/Price,5,"Good quality products, fast delivery, affordab...","Expensive, artificial sweeteners, loyalty syst..."
10,Customer Service,4,"Fast and serious, excellent delivery and commu...","Disconnected English support number, no answer..."
17,Quality & Price,3,"Good quality products, fast delivery, fair pri...","Inflation, high prices, no loyalty benefits Pr..."
24,Quality Products,3,"Fast delivery, high quality, recommended Excel...","Expensive, long wait, poor delivery service Ex..."


In [147]:
df_all_themes.to_excel("Nutrimuscle_Review_Themes.xlsx")

In [117]:
# Use OpenAI to embed the summaries and titles. Size of _embeds: (num_chunks x 1536)
openai_embed = OpenAIEmbeddings()

summary_embeds = np.array(openai_embed.embed_documents(stage_1_summaries))
title_embeds = np.array(openai_embed.embed_documents(stage_1_titles))

In [118]:
# Get similarity matrix between the embeddings of the chunk summaries
summary_similarity_matrix = np.zeros((num_1_chunks, num_1_chunks))
summary_similarity_matrix[:] = np.nan

for row in range(num_1_chunks):
  for col in range(row, num_1_chunks):
    # Calculate cosine similarity between the two vectors
    similarity = 1- cosine(summary_embeds[row], summary_embeds[col])
    summary_similarity_matrix[row, col] = similarity
    summary_similarity_matrix[col, row] = similarity

In [119]:
# Run the community detection algorithm

def get_topics(title_similarity, num_topics = 8, bonus_constant = 0.25, min_size = 3):

  proximity_bonus_arr = np.zeros_like(title_similarity)
  for row in range(proximity_bonus_arr.shape[0]):
    for col in range(proximity_bonus_arr.shape[1]):
      if row == col:
        proximity_bonus_arr[row, col] = 0
      else:
        proximity_bonus_arr[row, col] = 1/(abs(row-col)) * bonus_constant
        
  title_similarity += proximity_bonus_arr

  title_nx_graph = nx.from_numpy_array(title_similarity)

  desired_num_topics = num_topics
  # Store the accepted partitionings
  topics_title_accepted = []

  resolution = 0.85
  resolution_step = 0.01
  iterations = 40

  # Find the resolution that gives the desired number of topics
  topics_title = []
  while len(topics_title) not in [desired_num_topics, desired_num_topics + 1, desired_num_topics + 2]:
    topics_title = community.louvain_communities(title_nx_graph, weight = 'weight', resolution = resolution)
    resolution += resolution_step
  topic_sizes = [len(c) for c in topics_title]
  sizes_sd = np.std(topic_sizes)
  modularity = community.modularity(title_nx_graph, topics_title, weight = 'weight', resolution = resolution)

  lowest_sd_iteration = 0
  # Set lowest sd to inf
  lowest_sd = float('inf')

  for i in range(iterations):
    topics_title = community.louvain_communities(title_nx_graph, weight = 'weight', resolution = resolution)
    modularity = community.modularity(title_nx_graph, topics_title, weight = 'weight', resolution = resolution)
    
    # Check SD
    topic_sizes = [len(c) for c in topics_title]
    sizes_sd = np.std(topic_sizes)
    
    topics_title_accepted.append(topics_title)
    
    if sizes_sd < lowest_sd and min(topic_sizes) >= min_size:
      lowest_sd_iteration = i
      lowest_sd = sizes_sd
      
  # Set the chosen partitioning to be the one with highest modularity
  topics_title = topics_title_accepted[lowest_sd_iteration]
  print(f'Best SD: {lowest_sd}, Best iteration: {lowest_sd_iteration}')
  
  topic_id_means = [sum(e)/len(e) for e in topics_title]
  # Arrange title_topics in order of topic_id_means
  topics_title = [list(c) for _, c in sorted(zip(topic_id_means, topics_title), key = lambda pair: pair[0])]
  # Create an array denoting which topic each chunk belongs to
  chunk_topics = [None] * title_similarity.shape[0]
  for i, c in enumerate(topics_title):
    for j in c:
      chunk_topics[j] = i
            
  return {
    'chunk_topics': chunk_topics,
    'topics': topics_title
    }

In [120]:
# Set num_topics to be 1/4 of the number of chunks, or 8, which ever is smaller
num_topics = min(int(num_1_chunks / 4), 8)
topics_out = get_topics(summary_similarity_matrix, num_topics = num_topics, bonus_constant = 0.2)
chunk_topics = topics_out['chunk_topics']
topics = topics_out['topics']

Best SD: 5.2805584478027665, Best iteration: 9


In [54]:
def summarize_stage_2(stage_1_outputs, topics, summary_num_words = 250):
  print(f'Stage 2 start time {datetime.now()}')
  
  # Prompt that passes in all the titles of a topic, and asks for an overall title of the topic
  title_prompt_template = """Write an informative title that summarizes each of the following groups of titles. Make sure that the titles capture as much information as possible, 
  and are different from each other:
  {text}
  
  Return your answer in a numbered list, with new line separating each title: 
  1. Title 1
  2. Title 2
  3. Title 3

  TITLES:
  """

  map_prompt_template = """Wite a 10-50 word summary of the following text:
    {text}

    CONCISE SUMMARY:"""

  combine_prompt_template = 'Write a ' + str(summary_num_words) + """-word summary of the following, removing irrelevant information. Finish your answer:
  {text}
  """ + str(summary_num_words) + """-WORD SUMMARY:"""

  title_prompt = PromptTemplate(template=title_prompt_template, input_variables=["text"])
  map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])
  combine_prompt = PromptTemplate(template=combine_prompt_template, input_variables=["text"])

  topics_data = []
  for c in topics:
    topic_data = {
      'summaries': [stage_1_outputs[chunk_id]['summary'] for chunk_id in c],
      'titles': [stage_1_outputs[chunk_id]['title'] for chunk_id in c]
    }
    topic_data['summaries_concat'] = ' '.join(topic_data['summaries'])
    topic_data['titles_concat'] = ', '.join(topic_data['titles'])
    topics_data.append(topic_data)
    
  # Get a list of each community's summaries (concatenated)
  topics_summary_concat = [c['summaries_concat'] for c in topics_data]
  topics_titles_concat = [c['titles_concat'] for c in topics_data]

  # Concat into one long string to do the topic title creation
  topics_titles_concat_all = ''''''
  for i, c in enumerate(topics_titles_concat):
    topics_titles_concat_all += f'''{i+1}. {c}
    '''
  
  # print('topics_titles_concat_all', topics_titles_concat_all)

  title_llm = OpenAI(temperature=0, model_name = 'text-davinci-003')
  title_llm_chain = LLMChain(llm = title_llm, prompt = title_prompt)
  title_llm_chain_input = [{'text': topics_titles_concat_all}]
  title_llm_chain_results = title_llm_chain.apply(title_llm_chain_input)
  
  
  # Split by new line
  titles = title_llm_chain_results[0]['text'].split('\n')
  # Remove any empty titles
  titles = [t for t in titles if t != '']
  # Remove spaces at start or end of each title
  titles = [t.strip() for t in titles]

  map_llm = OpenAI(temperature=0, model_name = 'text-davinci-003')
  reduce_llm = OpenAI(temperature=0, model_name = 'text-davinci-003', max_tokens = -1)

  # Run the map-reduce chain
  docs = [Document(page_content=t) for t in topics_summary_concat]
  chain = load_summarize_chain(chain_type="map_reduce", map_prompt = map_prompt, combine_prompt = combine_prompt, return_intermediate_steps = True,
                              llm = map_llm, reduce_llm = reduce_llm)

  output = chain({"input_documents": docs}, return_only_outputs = True)
  summaries = output['intermediate_steps']
  stage_2_outputs = [{'title': t, 'summary': s} for t, s in zip(titles, summaries)]
  final_summary = output['output_text']

  # Return: stage_1_outputs (title and summary), stage_2_outputs (title and summary), final_summary, chunk_allocations
  out = {
    'stage_2_outputs': stage_2_outputs,
    'final_summary': final_summary
  }
  print(f'Stage 2 done time {datetime.now()}')
  
  return out

In [ ]:
# Query GPT-3 to get a summarized title for each topic_data
out = summarize_stage_2(stage_1_outputs, topics, summary_num_words = 250)
stage_2_outputs = out['stage_2_outputs']
stage_2_titles = [e['title'] for e in stage_2_outputs]
stage_2_summaries = [e['summary'] for e in stage_2_outputs]
final_summary = out['final_summary']

Stage 2 start time 2023-08-09 13:24:33.896892
Stage 2 done time 2023-08-09 13:24:51.528646
